##### <span style='color:royalblue'>The main goal of the mini-project is to build the database of restaurants, bars and various points of interest (POIs) in the area of your choice and find out which API has better coverage in the selected area. The APIs have limited number of requests for free, so start with the smaller area. The project consists of following tasks:</span>

In [2]:
import requests as rq
import pandas as pd
import json as js
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import os

# Setting up API keys
# YELP API setup
y_key = os.environ['YELP_KEY']
y_header = {'Authorization': 'Bearer '+y_key}
y_srch_url = 'https://api.yelp.com/v3/businesses/search'

# FSQ API setup
f_key = os.environ['FSQ_API_KEY']
f_header = {'Authorization': f_key, 'Accept': 'application/json'}
f_srch_url = 'https://api.foursquare.com/v3/places/search'

# Google Place API setup
g_key = '&key=' + os.environ['GPLACE_KEY']
g_srch_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

1. pull the data about various POIs in the area through API. (Search Yelp for companiees that are in the area using these instructions). If you run out of requests for any of the APIs, don't worry, it's ok to use only sample data you already have or the POIs from the Yelp API. It's approach and process that counts, not the actual number of places we were able to get.

<span style='color:blue'>Yelp needs detail search to get ratings, 50 results/page, 66 total results</span>

In [99]:
# YELP search for Kerrisdale Restaurants
y_param = {
    'location': '49.234456,-123.155144',
    'radius': '1000',
    'categories': 'restaurants',
    'limit': '50'
}
y_res = rq.get(y_srch_url, params=y_param, headers=y_header).json()
with open('restaurants_kerri_yelp.json', 'w') as f:
    f.write(js.dumps(y_res, indent=4))

In [101]:
y_param = {
    'location': '49.234456,-123.155144',
    'radius': '1000',
    'categories': 'Restaurants',
    'limit': '50',
    'offset': '50'
}
y_res = rq.get(y_srch_url, params=y_param, headers=y_header).json()
with open('restaurants_kerri_yelp.json', 'a') as f:
    f.write(js.dumps(y_res, indent=4))

In [33]:
# YELP details
with open('restaurants_kerri_yelp.json', 'r') as f:
    y_base = js.load(f)
y_id = []
for y in y_base['businesses']:
    y_id.append(y['id'])

with open('restaurants_kerri_yep.json', 'a') as f:
    for i in y_id:
        y_detl_url = f'https://api.yelp.com/v3/businesses/{i}'
        y_re2 = rq.get(y_detl_url, headers=y_header).json()
        f.write(js.dumps(y_re2, indent=4))


#### <span style='color:darkgreen'>Extracting Yelp Data</span>

In [3]:
with open('restaurants_kerri_yep.json', 'r') as f:
    y_data = js.load(f)
df = pd.json_normalize(y_data, record_path='businesses', errors='ignore')
df_y0 = df[['name', 'rating', 'price', 'hours', 'location.address1', 'display_phone', 'review_count']].copy()
df_y0.rename(columns={'hours': 'hours_', 'location.address1': 'address', 'display_phone': 'phone_number'}, inplace=True)

In [4]:
days = ['M', 'T', 'W', 'R', 'F', 'S', 'U']
hours = []
for i in df_y0['hours_']:
    hl = []
    if pd.isnull(i):
        hours.append('')
        continue
    for j in i[0]['open']:
        ts = dt.datetime.strptime(j['start'], '%H%M').strftime('%H:%M')
        te = dt.datetime.strptime(j['end'], '%H%M').strftime('%H:%M')
        td = days[j['day']]
        tt = ts+'-'+te+' '+td
        hl.append(tt)
    hours.append(hl)

df_y = df_y0.copy()
df_y.insert(3, 'hours', hours)
df_y.drop('hours_', axis=1, inplace=True)
df_y.fillna('N/A', inplace=True)
df_y.sort_values(by=['rating', 'review_count'], ascending=False).head(10)

,name,rating,price,hours,address,phone_number,review_count
57,Sushi time,5.0,N/A,"[11:00-21:00 M, 11:00-21:00 T, 11:00-21:00 W, ...",5351 West Blvd,+1 604-264-7555,2
7,Sweet E's Pastries And Sweets,4.5,$$,"[11:30-16:00 T, 11:30-16:00 W, 11:30-16:00 R, ...",2032 West 41st Avenue,+1 604-264-9202,26
26,Castella Cheese Tart,4.5,N/A,"[13:30-21:30 M, 13:30-21:30 T, 13:30-21:30 W, ...",2184 W 41st Avenue,,12
59,Quilchena Off-leash Dog Park,4.5,N/A,,33rd and Pine Crescent,,3
0,Ajisai Sushi Bar,4.0,$$,"[11:00-19:30 T, 11:00-19:30 W, 11:00-19:30 R, ...",2081 West 42nd Avenue,+1 604-266-1428,388
1,Nana's Green Tea,4.0,$$,"[11:00-21:00 M, 11:00-21:00 T, 11:00-21:00 W, ...",2135 W 41st Avenue,+1 604-263-5054,155
5,Sofra Mediterranean Kitchen,4.0,$$,"[11:00-19:30 M, 11:00-19:30 T, 11:00-19:30 W, ...",2277 W 41st Ave,+1 604-558-3287,91
14,Minerva Restaurant,4.0,$$,"[16:00-23:00 T, 16:00-23:00 W, 16:00-23:00 R, ...",2411 W 41st Avenue,+1 604-263-1774,88
9,Sip Bowl La Mian,4.0,$$,"[11:00-21:00 M, 11:00-21:00 T, 11:00-21:00 W, ...",2255 W 41 Ave,+1 778-737-3999,70
3,Honolulu Coffee,4.0,$$,"[08:30-17:00 M, 08:30-17:00 T, 08:30-17:00 W, ...",2098 W 41 Avenue,+1 604-620-2168,64


<span style='color:blue'>FSQ, 50/page, 99 results found</span>

In [132]:
# FOURSQUARE search for Kerrisdale Restaurants
f_param = {
    'll': '49.234456,-123.155144',
    'radius': '1000',
    'categories': '13065',
    'fields': 'name,geocodes,tel,location,categories,distance,website,hours,rating,popularity,price',
    'limit': '50'
}
f_res = rq.get(f_srch_url, params=f_param, headers=f_header)
with open('restaurants_kerri_fsq.json', 'w') as f:
    f.write(js.dumps(f_res.json(), indent=4))

In [133]:
f_res.headers['Link']

'<https://api.foursquare.com/v3/places/search?cursor=c3I6NTA&radius=1000&ll=49.234456%2C-123.155144&fields=name%2Cgeocodes%2Ctel%2Clocation%2Ccategories%2Cdistance%2Cwebsite%2Chours%2Crating%2Cpopularity%2Cprice&categories=13065&limit=50>; rel="next"'

In [134]:
f_srch_url_np = 'https://api.foursquare.com/v3/places/search?cursor=c3I6NTA&radius=1000&ll=49.234456%2C-123.155144&fields=name%2Cgeocodes%2Ctel%2Clocation%2Ccategories%2Cdistance%2Cwebsite%2Chours%2Crating%2Cpopularity%2Cprice&categories=13065&limit=50&rel=next'
f_re2 = rq.get(f_srch_url_np, headers=f_header)
with open('restaurants_kerri_fsq.json', 'a') as f:
    f.write(js.dumps(f_re2.json(), indent=4))

#### <span style='color:darkgreen'>Extracting FSQ Data</span>

In [5]:
with open('restaurants_kerri_fsq.json', 'r') as f:
    f_data = js.load(f)
df = pd.json_normalize(f_data, record_path='results', errors='ignore')
df_f0 = df[['name', 'rating', 'price', 'hours.display', 'location.address', 'tel', 'website']].copy()
df_f0.rename(columns={'hours.display': 'hours', 'location.address': 'address', 'tel': 'phone_number'}, inplace=True)

In [6]:
df_f = df_f0.copy()
df_f['price'].fillna(0, inplace=True)
df_f['rating'].fillna(0, inplace=True)
for i in range(len(df_f['price'])): df_f['price'][i] = str('$' * int(df_f['price'][i]))
# for i in range(len(df_f['rating'])): df_f['rating'][i] = round(df_f['rating'][i]/2.0, 1)
df_f.fillna('N/A', inplace=True)
df_f.sort_values(by=['rating'], ascending=False).head(10)

/tmp/ipykernel_21643/1506117380.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for i in range(len(df_f['price'])): df_f['price'][i] = str('$' * int(df_f['price'][i]))


,name,rating,price,hours,address,phone_number,website
0,Ajisai Sushi Bar,8.9,$$,"Tue-Sat 11:30-14:00, 17:00-21:30; Sun 17:00-21:30",2081 42nd Ave W,(604) 266-1428,http://www.sushibarajisai.com/
3,Minerva Pizza & Steak House,8.5,$$$,Mon-Sat 16:00-23:00; Sun 16:00-22:00,2411W 41st Ave W,(604) 263-1774,http://www.minervas.ca
1,Nando's Chicken Kerrisdale,8.5,$$,Mon-Thu 11:00-22:00; Fri-Sat 11:00-22:30; Sun ...,2064W 41st Ave W,(604) 266-4485,http://www.nandos.ca/
2,Golden Ocean Seafood Restaurant,8.3,$,"Open Daily 8:00-15:00, 17:00-22:00",2046 41 St Ave W,(604) 263-8886,N/A
4,Pizzeria Bufala,8.1,$,Open Daily 11:30-22:00,5395 West Blvd,(604) 267-7499,http://www.bufala.ca
5,Secret Garden Tea Co,7.7,$$,Mon-Sat 8:00-18:30; Sun 9:00-18:30,2138 40th Ave W,(604) 261-3070,http://www.secretgardentea.com
6,White Spot,7.7,$$,Mon-Thu 6:30-22:00; Fri-Sat 6:30-23:00; Sun 6:...,5367 West Blvd,(604) 266-1288,https://www.whitespot.ca/
7,Bistro 41 by Faubourg,7.4,$,Mon 7:00-18:00; Tue-Sat 7:00-19:00; Sun 8:00-1...,2156 41st Ave W,(604) 266-2156,http://www.faubourg.com
17,Caffe Artigiano,7.2,$,Mon-Thu 6:30-18:00; Fri-Sat 6:30-19:00; Sun 6:...,2154W 41st Ave W,(604) 267-1008,http://www.caffeartigiano.com
8,Pho Tan's Vietnamese Beef Noodle Soup & Bubble...,7.2,$$,Open Daily 10:00-21:00,2076W 41st Ave W,(604) 606-0688,N/A


Google Place nearby search  
<span style='color:red'> returns 20 results/page, needs to use next_page_token for additional results (corporation greed at play?) Only 60 results max </span>

In [52]:
# Google Place nearby search
g_param = {
    'location': '49.234456,-123.155144',
    'rankby': 'distance',
    'type': 'restaurant'
}
g_res = rq.get(g_srch_url+g_key, params=g_param)
with open('restaurants_kerri_ggpl.json', 'w') as f:
    f.write(js.dumps(g_res.json(), indent=4))

In [55]:
g_nextpage = 'Aap_uEBI25qXTczBFoDFUA7ZdUFm12AQivOTvRA42ROhbCjHMaKssiRw_zjaF6JpgkUkomxvbf5zVYL7p2an1HkUF9sdXeJKaAgl0umB-9xSwY3oqmGD-p2TV6y9qKz3L89xqOvRoUG75pWEJUzYFDRBZPWCuHTaJ0NgwHzEhX-H0WjvoN-tBFRMuIWtOJJuR8u_W8jIbz298QjFG6FBLXHdP1vzVn-W0EWMhwAU7A-7PTNj_PcQoBppx29x0okRZ7ap7SP2B8fQXFi2EbVe2KiqS7A6u7FDYlx6xI2mWLmLwa6zUJZeaEb89GLPvfIqJYdEPlDin8okjpJJ7PIMqZpist9D-N2mxwSjJdAK6ADrgYnqTmV-vMKHjBVgnxyv2ZYYlighRME92v3h_JF0PKxyiEDNcV6GRB8YzFiGXsR9ryngQ0__KGxmOkwd1xYgNBSHOlZb2LRWUBGdgQNQz2CsVYaqaJK433QQkRcFHz3GKwW4xdFeezSM-q4Lv_xLQZJ2IwpZbK0mDa-IaJ09FXXQRnex_sYmzlPUiF173JEVVuJmBhtdVjm4Fun2_cAX8Z_5lHzzHqgmg7mhVPfa79OVBa9N-TDZ2UEh5SzRZmbReGwEQVJGs4PyDP7kdLBLltrAXTqXlZm8i_CEgDuhkuMk'
g_srch_url_np = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken={g_nextpage}'
g_re2 = rq.get(g_srch_url_np+g_key)
with open('restaurants_kerri_ggpl.json', 'a') as f:
    f.write(js.dumps(g_re2.json(), indent=4))

In [171]:
# Google Place details
with open('restaurants_kerri_ggpl.json', 'r') as f:
    g_base = js.load(f)
g_id = []
for g in g_base['results']:
    g_id.append(g['place_id'])

g_params = {
    'fields': 'name,adr_address,formatted_phone_number,geometry,opening_hours,price_level,rating,types,vicinity,website'
}
with open('restaurants_kerri_gpl.json', 'w') as f:
    for i in g_id:
        g_detl_url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={i}'
        g_re2 = rq.get(g_detl_url+g_key, params=g_params).json()
        f.write(js.dumps(g_re2, indent=4))

#### <span style='color:darkgreen'>Extracting GooglePlace Data</span>

In [7]:
with open('restaurants_kerri_gpl.json', 'r') as f:
    g_data = js.load(f)
df = pd.json_normalize(g_data, record_path='results', errors='ignore')
df_g0 = df[['name', 'rating', 'price_level', 'opening_hours.weekday_text', 'adr_address', 'formatted_phone_number', 'website']].copy()
df_g0.rename(columns={'price_level': 'price', 'opening_hours.weekday_text': 'hours', 'adr_address': 'address', 'formatted_phone_number': 'phone_number'}, inplace=True)

In [8]:
df_g = df_g0.copy()
df_g['price'].fillna(0, inplace=True)
for i in range(len(df_g['price'])): df_g['price'][i] = str('$' * int(df_g['price'][i]))
# df_g.insert(3, 'hours', hours)
# df_g.drop('hours_', axis=1, inplace=True)
df_g.fillna('N/A', inplace=True)
df_g.sort_values(by=['rating'], ascending=False).head(10)

/tmp/ipykernel_21643/3772272318.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for i in range(len(df_g['price'])): df_g['price'][i] = str('$' * int(df_g['price'][i]))


,name,rating,price,hours,address,phone_number,website
21,Hanppy to go vancouver,5.0,,"[Monday: 11:00 AM-9:00 PM, Tuesday: 11:00 AM-9...",5555 West Blvd,(604) 559-9488,http://hanppytofupot.com/
18,Pachamami Foodtruck,5.0,,N/A,5900 West Blvd,N/A,N/A
40,Sushi Time,4.9,,"[Monday: Closed, Tuesday: 11:00 AM-9:00 PM, We...",5351 West Blvd,(604) 264-7555,http://www.sushi-time.ca/
54,BIGSBY THE BAKEHOUSE,4.6,$$,"[Monday: Closed, Tuesday: 10:00 AM-4:00 PM, We...",4894 MacKenzie St,(604) 267-2253,http://bigsbybake.house/
57,Land & Sea Restaurant & Bar,4.6,,"[Monday: Closed, Tuesday: 5:00-10:00 PM, Wedne...",1864 W 57th Ave,(604) 263-0155,http://www.landandseayvr.com/
33,Kokoro Tokyo Mazesoba - Kerrisdale,4.5,,"[Monday: 11:30 AM-8:30 PM, Tuesday: 11:30 AM-8...",2285 W 41st Ave,(604) 326-2965,http://menyakokoro.com/
47,Hikari Cafe,4.5,,"[Monday: Closed, Tuesday: 2:00-8:00 PM, Wednes...",5701 Granville St,(604) 269-3682,http://hikaricafe.ca/
50,Tao's Kitchen,4.5,,"[Monday: 5:00-9:30 PM, Tuesday: Closed, Wednes...",5701 Granville St #110,(778) 379-2218,N/A
8,Sushi Bar Ajisai,4.5,$$,"[Monday: Closed, Tuesday: 12:00-7:30 PM, Wedne...",2081 W 42nd Ave,(604) 266-1428,https://sushibarajisai.com/
14,Baan Wasana Thai Restaurant,4.4,$,"[Monday: Closed, Tuesday: 11:00 AM-2:30 PM, 4:...",2143 W 41st Ave,(778) 371-8971,http://www.baanwasana.com/


2. create own SQLite database and store the data about the POIs. Think about what will be the best structure of the database. We've used and created sqlite3 databases before in the activity SQL in Python.

In [9]:
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection



#### <span style='color:red'> `pandas.DataFrame.to_sql()` when converting sqlite3, conver all obj in df to sqlite3 recognizing types such as TEXT(string), INT(int64, int32), REAL(float), NULL(Nan) ...

In [ ]:
con_y = create_connection('kerrisdale_rest_yelp.db')

df_y_db = df_y.astype({'name': 'string', 'price': 'string', 'hours': 'string', 'address': 'string', 'phone_number': 'string'}).copy()
df_y_db.to_sql('rest_yelp', con=con_y)

In [13]:
con_f = create_connection('kerrisdale_rest_fsq.db')
# df_f.dtypes
df_f_db = df_f.astype({'name': 'string', 'price': 'string', 'hours': 'string', 'address': 'string', 'phone_number': 'string', 'website': 'string'}).copy()
df_f_db.to_sql('rest_fsq', con=con_f)

Connection to SQLite DB successful


99

In [14]:
con_g = create_connection('kerrisdale_rest_gpl.db')
# df_g.dtypes
df_g_db = df_g.astype({'name': 'string', 'price': 'string', 'hours': 'string', 'address': 'string', 'phone_number': 'string', 'website': 'string'}).copy()
df_g_db.to_sql('rest_gpl', con=con_g)

Connection to SQLite DB successful


60

3. a) compare the results using SQL or Pandas (it's up to you:)) and see which API has a better coverage of the area.  
b) choose the top 10 POIs based on the popularity (number of reviews or average rating) (Yelp, Foursquare).

4. (Stretch) By implementing travelling salesman problem (TSP), how much time would it take to visit all of these places? (Directions API from google will be helpful here). We will have to find travel time between all places (top 10). We can use ortools from Google to effectively implement TSP. These tools are very powerful and easy to install.

In [216]:
g_tt = pd.json_normalize(g_base, record_path='results', errors='ignore').sort_values(by=['rating'], ascending=False).head(10)
g_pid = g_tt['place_id'].to_list()
g_dst = ''

# Distance Matrix API
g_key = '&key=' + os.environ['GPLACE_KEY']
g_dist_url = 'https://maps.googleapis.com/maps/api/distancematrix/json?'


for i in range(len(g_pid)):
    g_pid[i] = 'place_id:'+g_pid[i]+'|'
    g_dst += g_pid[i]
g_dst = g_dst[:-1]

for i in range(len(g_pid)):
    g_dparam = {
        'origins': g_pid[i][:-1],
        'destinations': g_dst,
        'mode': 'walking'
    }
    g_dre = rq.get(g_dist_url+g_key, params=g_dparam)
    with open('restaurants_kerri_dmat.json', 'a') as f:
        f.write('\n')
        g_js = js.loads(g_dre.text)
        g_row = []
        for i in g_js['rows'][0]['elements']:
            g_row.append(i['distance']['value'])
        for i in g_row:
            f.write(str(i)+',')

In [218]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [0,323,196,1782,2009,379,1289,1363,333,244],
        [323,0,519,2090,1691,473,1278,1334,205,247],
        [196,519,0,1586,2204,574,1485,1559,528,440],
        [1782,2090,1586,0,3779,1628,3069,3071,2103,1855],
        [2009,1691,2204,3779,0,2165,2233,2244,1808,1931],
        [379,473,574,1626,2165,0,1446,1520,486,239],
        [1289,1278,1485,3069,2233,1446,0,74,1067,1213],
        [1363,1334,1559,3143,2242,1520,74,0,1141,1287],
        [333,205,528,2103,1808,486,1067,1141,0,257],
        [244,247,440,1855,1931,239,1213,1287,257,0],
    ]  # yapf: disable
    data['num_vehicle'] = 1
    data['depot'] = 0
    return data


def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} meters'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {} meters\n'.format(route_distance)


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicle'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)


if __name__ == '__main__':
    main()

Objective: 9303 meters
Route for vehicle 0:
 0 -> 2 -> 3 -> 5 -> 9 -> 8 -> 6 -> 7 -> 4 -> 1 -> 0

